---
title: "HMC vs Pólya-Gamma Sampler"
subtitle: "Julia によるベイズ・ロジスティック回帰での計算効率比較"
author: "司馬 博文"
date: 12/3/2025
categories: [Computation, MCMC]
# image: Images/Cochran.svg
bibliography: 
    - ../../../assets/2023.bib
    - ../../../assets/2024.bib
    - ../../../assets/2025.bib
csl: ../../../assets/apalike.csl
# jupyter: julia-1.12
code-fold: false
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\bR}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\operatorname{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\operatorname{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



ベイズ・ロジスティック回帰において，MCMC により事後分布サンプリングを行う方法はいくつかある：

* Pólya-Gamma 拡大に基づく Gibbs サンプラー [@Polson+2013]
* HMC (Hamiltonian Monte Carlo) [@Duane+1987]

本稿ではこの２つのサンプラーの性能を比較する．

他にも，Zig-Zag Sampler with Control Variate [@Bierkens+2019], Zig-Zag Sampler with Importance Resampling [@Sen+2020] などの方法が提案されており，大標本 $n\gg1$ に対するスケーラビリティが期待されるが，ここでは扱わない．

どちらかというと，中程度の次元 $d\approx50$ にスパース性を仮定した設定の下で，サンプラーの性能を比較することを考える．


In [ ]:
#| output: false
n, p, pₑ = 200, 50, 10

using Random, StatsFuns, Distributions
β_true = vcat(randn(pₑ), zeros(p - pₑ))
X = randn(n, p)

η_true = X * β_true
π_true = logistic.(η_true)

y = rand.(Bernoulli.(π_true))
y = collect(Float64, y)

以降，次のように進む

1. Pólya-Gamma 拡大を解説し，[`PolyaGammaHybridSamplers.jl`](https://github.com/wzhorton/PolyaGammaHybridSamplers.jl) パッケージを利用して Gibbs サンプラーを実装する．
2. [`Turing.jl`](https://turinglang.org/) を用いて HMC サンプラーを実装する方法を解説する．Turing は完全 Julia ベースの確率的プログラミング言語である．このフレームワーク上で比較することで，Julia 言語の JIT コンパイル，型チェック，ガベージコレクションなどの条件をなるべく揃えた比較を狙う．
3. 最後に次の３点を指標にして，２つのサンプラーを比較検討する．
  
    1. 事後平均により真の回帰係数 $\beta$ を推定したときの $\ell^2$-誤差
    2. 3000 サンプルあたりの有効サンプル数 (ESS: 独立サンプル何個分に当たるか)
    3. 単位実行時間あたりの有効サンプル数

これら３つの指標に対して，HMC / PG サンプラーはそれぞれ得意・不得意が違う．

一見 HMC の方がサンプルの質が良いが，実は計算時間で割ると PG サンプラーの方が効率が良いことを見る．

従って，我々の設定（中程度の次元 $d\approx50$ にスパース性を仮定した設定で Gaussian sample of size $n=200$）においては，PG サンプラーからより多くのサンプルを抽出して Monte Carlo 推定量を構成した方が，分散は小さくなると予想される．

## Pólya-Gamma 拡大に基づく Gibbs サンプラー

Pólya-Gamma 拡大に基づく Gibbs サンプラーは，次のように定義される：


In [ ]:
#| output: false
using PolyaGammaHybridSamplers, LinearAlgebra, MCMCChains, Dates, MCMCDiagnosticTools

function pg_logistic_gibbs(
  X::Matrix{Float64},
  y::Vector{Float64};
  n_samples::Int = 5000,
  burnin::Int = 1000,
  σ_prior::Float64 = 10.0,
)
  n, p = size(X)

  # 事前: β ~ N(0, σ_prior^2 I)
  V0_inv = (1 / σ_prior^2) * LinearAlgebra.I  # precision of prior

  # 初期値
  β = zeros(p)
  κ = y .- 0.5  # κ_i = y_i - 1/2

  # サンプル保存用
  n_iter = n_samples + burnin
  β_samples = Matrix{Float64}(undef, n_samples, p)

  t_start = time()
  for it in 1:n_iter
    # 1. PG 補助変数 ω_i | β のサンプル
    η = X * β
    ω = similar(η)
    for i in 1:n
      pg = PolyaGammaHybridSampler(1.0, η[i])
      ω[i] = rand(pg)
    end

    # 2. β | ω, y のサンプル (多変量ガウス)
    Ω = Diagonal(ω)
    precision = X' * Ω * X + V0_inv          # posterior precision
    cov = inv(Matrix(precision))             # posterior covariance
    m = cov * (X' * κ)                       # posterior mean (μ0=0 のため)

    # β ~ N(m, cov)
    β = rand(MvNormal(m, Symmetric(cov)))

    # burn-in 後に保存
    if it > burnin
      β_samples[it - burnin, :] .= β
    end
  end
  t_stop = time()
  runtime_sec = t_stop - t_start

  names = Symbol.("β[$i]" for i in 1:p)
  values = reshape(β_samples, n_samples, p, 1)
  chain = Chains(values, names)
  chain = setinfo(chain, (
    start_time = [t_start],  # 1本チェインなら長さ1のベクトルでOK
    stop_time  = [t_stop],
  ))

  return chain, runtime_sec
end

In [ ]:
σ_prior = 10.0
chain_pg, t_pg = pg_logistic_gibbs(X, y;
    n_samples = 3000,
    burnin = 3000,
    σ_prior = σ_prior,
)
summarize(chain_pg)

## HMC


In [ ]:
#| output: false
using Turing, LinearAlgebra

@model function logreg_turing(x, y, σ_prior)
    n, p = size(x)
    
    # 事前分布
    β ~ MvNormal(zeros(p), (σ_prior^2) * I)
    
    # ベクトル化した尤度（高速化）
    η = x * β
    y ~ arraydist(Bernoulli.(logistic.(η)))
end

model = logreg_turing(X, y, σ_prior)

In [ ]:
#| output: false
n_samples = 3000
n_adapt   = 3000

chain_hmc = sample(
    model,
    NUTS(n_adapt, 0.6),
    n_samples,
)

In [ ]:
summarize(chain_hmc)

## 結果の比較


In [ ]:
using Statistics

# 真の β との誤差
mean_hmc = vec(mean(Array(chain_hmc), dims=1))  # ここは実際のパラメータ名に合わせて調整
mean_pg = vec(mean(Array(chain_pg), dims=1))

println("‖β̂_HMC - β_true‖₂ = ", norm(mean_hmc .- β_true))
println("‖β̂_PG  - β_true‖₂ = ", norm(mean_pg  .- β_true))

# ランタイムや ESS の比較も：
ess_hmc = ess_rhat(chain_hmc)
ess_pg  = ess_rhat(chain_pg)
println("ESS (HMC) = ", mean(ess_hmc[:,:ess]))
println("ESS (PG) = ", mean(ess_pg[:,:ess]))
println("ESS/s (HMC) = ", mean(ess_hmc[:,:ess_per_sec]))
println("ESS/s (PG) = ", mean(ess_pg[:,:ess_per_sec]))